In [2]:
!pip install groq
!pip install openai
!pip install together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.4/360.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 37.9 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0


In [8]:
from groq import Groq
from together import Together


groq_client = Groq(api_key=("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"))
together_client = Together(api_key=("b2ceded2a0405d5f78a6d31e0004fee9f23c073fc2c05926be188188da0570a7"))


Available tasks:
1. cold email
2. market research report
3. product pitch
4. competitive analysis
Choose a task number: 1
Please enter the industry or company you want to target for cold email: tech startup
Generated queries: ['site:linkedin.com "tech startup" AND "founder" OR "co-founder"', 'site:crunchbase.com "tech startup" AND "seed funding" OR "series a"', 'site:angel.co "tech startup" AND "software" OR "SaaS"', 'site:owler.com "tech startup" AND "CEO" OR "CTO"']
Searching for... site:linkedin.com "tech startup" AND "founder" OR "co-founder"
Searching for... site:crunchbase.com "tech startup" AND "seed funding" OR "series a"
Searching for... site:angel.co "tech startup" AND "software" OR "SaaS"
Searching for... site:owler.com "tech startup" AND "CEO" OR "CTO"

Generated cold email:

Subject: Boost Your Tech Startup's Growth with Proven Strategies

Hi [Founder's Name],

As a fellow tech enthusiast, I understand the challenges of navigating the competitive startup landscape. Despite

In [ ]:

def get_user_input(prompt):
    return input(prompt)

def query_agent(target, task):
    response = together_client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo",
        messages=[
            {
                "role": "system",
                "content": f"""You are an AI assistant that writes concise search queries for {task}.
                Create 4 short search queries based on the given target industry or company.
                IMPORTANT: Respond with only the queries, one per line."""
            },
            {
                "role": "user",
                "content": f"Here's the industry / company to perform research on: #### {target} ####"
            }
        ],
        max_tokens=150,
        temperature=0.7,
        top_p=1,
        top_k=50,
        repetition_penalty=1,
        stop=["<|eot_id|>"]
    )
    queries = response.choices[0].message.content.split('\n')
    return [query.strip() for query in queries if query.strip()]



In [ ]:
def web_search_agent(query):
    # Simulate search results based on the query
    search_results = {
        "artificial intelligence market trends": "Artificial intelligence is growing rapidly, with increasing adoption across various industries. Key trends include advancements in machine learning, natural language processing, and computer vision.",
        "top ai companies to watch": "Some of the top AI companies to watch include OpenAI, DeepMind, Google AI, Microsoft AI, and IBM Watson.",
        "how to implement ai in business": "To implement AI in business, companies should identify use cases, gather relevant data, choose appropriate AI models, and integrate AI into existing workflows. It's also important to consider ethical considerations and ensure responsible AI development.",
        "benefits of using ai in business": "Using AI in business can provide benefits such as improved decision-making, increased efficiency, cost savings, and enhanced customer experience. AI can automate repetitive tasks, provide insights from data, and enable personalization at scale.",
    }

    if query in search_results:
        return search_results[query]
    else:
        return "No relevant search results found."

def task_agent(target, search_results, task):
    combined_results = "\n".join(search_results)

    response = together_client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo",
        messages=[
            {
                "role": "system",
                "content": f"""You are an expert {task} writer.
                Your task is to write a {task} based on the research given to you.
                Make sure to utilize all areas of the research provided.
                Keep the output concise and use plain English.
                DO NOT OUTPUT ANY OTHER TEXT !!! ONLY THE {task.upper()} ITSELF!."""
            },
            {
                "role": "user",
                "content": f"Here is the target: {target} \n Here is the research: #### {combined_results} #### ONLY OUTPUT THE {task.upper()} ITSELF. NO OTHER TEXT!!"
            }
        ],
        max_tokens=500,
        temperature=0.1,
        top_p=1,
        top_k=50,
        repetition_penalty=1,
        stop=["<|eot_id|>"]
    )
    return response.choices[0].message.content



In [ ]:
def main():
    tasks = ["cold email", "market research report", "product pitch", "competitive analysis"]
    print("Available tasks:")
    for i, task in enumerate(tasks, 1):
        print(f"{i}. {task}")

    task_choice = int(get_user_input("Choose a task number: ")) - 1
    chosen_task = tasks[task_choice]

    target = get_user_input(f"Please enter the industry or company you want to target for {chosen_task}: ")

    generated_queries = query_agent(target, chosen_task)
    print("Generated queries:", generated_queries)

    search_results = []
    for query in generated_queries:
        print("Searching for...", query)
        result = web_search_agent(query)
        search_results.append(result)

    final_output = task_agent(target, search_results, chosen_task)
    print(f"\nGenerated {chosen_task}:\n")
    print(final_output)

if __name__ == "__main__":
    main()